In [35]:
import sys
import logging
import json
import numpy as np
import pandas as pd
from datetime import datetime
import time

In [ ]:
# Integrated into sext_tools

In [36]:
dfknobs = pd.read_json(f'dfknobs_6and3.json')

In [43]:
dfknobs

,SVD0,SH1,SH3,SH4,SL1,SL2,SL3
SL1,0.000000,0,0,0,1,0,0
SL2,0.000000,0,0,0,0,1,0
SL3,0.000000,0,0,0,0,0,1
SH1,0.000000,1,0,0,0,0,0
SH3,0.000000,0,1,0,0,0,0
SH4,0.000000,0,0,1,0,0,0
SM1A,-0.663666,0,0,0,0,0,0
SM1B,0.744816,0,0,0,0,0,0
SM2B,-0.069260,0,0,0,0,0,0


In [37]:
sys.path.append('../')
import sext_tools
MODES = sext_tools.MODES    

In [38]:
dfdict = {}
for m in MODES:
    index = sext_tools.MODE_GROUPS[m].copy()
    indexnosvd = index.copy()
    columns = sext_tools.MODE_GROUPS[m].copy()
    if 'SVD0' in columns:
        index.remove('SVD0')
        indexnosvd.remove('SVD0')
        index.extend(['SM1A', 'SM1B', 'SM2B'])
    else:
        pass
    df = pd.DataFrame(np.zeros((len(index), len(columns))), index=index, columns=columns)
    if 'SVD0' in df.columns:
        df.loc[['SM1A', 'SM1B', 'SM2B'],'SVD0'] = dfknobs.loc[['SM1A', 'SM1B', 'SM2B'],'SVD0']
    for i in indexnosvd:
        df.loc[i,i] = 1.0

    if 'SVD0' in df.columns:
        # impact of K2L difference
        df.loc['SM2B',:] *= 0.25/0.2

    dfdict[m] = df

In [39]:
dfdict['BARE_SH1_SH3_SH4_SL1_SL2_SL3_SVD0']

,SH1,SH3,SH4,SL1,SL2,SL3,SVD0
SH1,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH3,0.0,1.0,0.0,0.0,0.0,0.0,0.000000
SH4,0.0,0.0,1.0,0.0,0.0,0.0,0.000000
SL1,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
SL2,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
SL3,0.0,0.0,0.0,0.0,0.0,1.0,0.000000
SM1A,0.0,0.0,0.0,0.0,0.0,0.0,-0.663666
SM1B,0.0,0.0,0.0,0.0,0.0,0.0,0.744816
SM2B,0.0,0.0,0.0,0.0,0.0,0.0,-0.086576


In [40]:
dfdict['BARE_SH1P12345_SH3_SH4_SL1_SL2_SL3_SVD0']

,SH1-P1,SH1-P2,SH1-P3,SH1-P4,SH1-P5,SH3,SH4,SL1,SL2,SL3,SVD0
SH1-P1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH1-P2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH1-P3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH1-P4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH1-P5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
SH3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000
SH4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000
SL1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
SL2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
SL3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000


In [41]:
dump = {m:df.to_json() for m,df in dfdict.items()}

In [42]:
with open(f'dfknobs_all.json', 'w') as f:
    json.dump(dump, f)